# TP4 - DÉTECTION DE FRAUDE BANCAIRE
## Optimisation ML & Feature Engineering Avancé

**Master 1 Data Engineering - Concepts & Techno IA**
**Durée : 7 heures**
**Nom & Prénom : PIBRE Alec**

---

### Objectifs
- Maîtriser le Feature Engineering avancé
- Gérer des données fortement déséquilibrées (0.17% de fraudes)
- Optimiser des hyperparamètres avec GridSearchCV
- Construire des Pipelines ML complets
- Analyser la performance (ROC, Learning Curves, Feature Importance)

### Mission
Développer un modèle de détection de fraude avec :
- **Recall ≥ 0.85** (détecter 85% des fraudes)
- **Precision maximale** (minimiser les faux positifs)
- **Explicabilité** (Feature Importance)

---

## IMPORTS & CONFIGURATION

In [ ]:
# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration visualisation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print(" Imports réussis")

In [ ]:
# Imports ML
from sklearn.model_selection import (
 train_test_split, StratifiedKFold, TimeSeriesSplit,
 GridSearchCV, cross_val_score, learning_curve
)
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Métriques
from sklearn.metrics import (
 classification_report, confusion_matrix, ConfusionMatrixDisplay,
 roc_auc_score, average_precision_score, precision_recall_curve,
 roc_curve, auc, f1_score, precision_score, recall_score
)

# Gestion déséquilibre
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

print(" Imports réussis")

---
## PARTIE 1 : EXPLORATION & FEATURE ENGINEERING (2h)
---

### 1.1 Chargement des Données

In [ ]:
# TODO: Charger le dataset creditcard.csv depuis le dossier ../data/



df = pd.read_csv('../data/creditcard.csv')


print(f"Shape: {df.shape}")
df.head()

In [ ]:
# TODO: Afficher les informations du dataset

print("=== Informations générales ===")
df.info()

In [ ]:
print("\n=== Dimensions ===")
print("Nombre de lignes et colonnes :", df.shape)

In [ ]:
print("\n=== Colonnes ===")
print(df.columns.tolist())

In [ ]:
print("\n=== Aperçu des premières lignes ===")
display(df.head())

In [ ]:
print("\n=== Statistiques descriptives ===")
display(df.describe())

### 1.2 Analyse du Déséquilibre

In [ ]:
# TODO: Calculer et afficher :
# - Nombre de transactions légitimes (Class = 0)
# - Nombre de fraudes (Class = 1)
# - Pourcentage de fraudes
# - Ratio fraudes/légitimes

legit_count = df[df['Class'] == 0].shape[0]
fraud_count = df[df['Class'] == 1].shape[0]
fraud_percentage = (fraud_count / len(df)) * 100

print(f"Transactions légitimes: {legit_count:,}")
print(f"Fraudes: {fraud_count:,}")
print(f"Pourcentage de fraudes: {fraud_percentage:.3f}%")
print(f"Ratio (1 fraude pour X légitimes): 1:{legit_count/fraud_count:.0f}")

In [ ]:
# TODO: Visualiser la distribution des classes avec un bar plot

n_legit = legit_count
n_fraud = fraud_count

labels = ['Légitimes (0)', 'Fraudes (1)']
values = [n_legit, n_fraud]
colors = ["#00F5A0", "#FF4D6D"]

total = n_legit + n_fraud
percentages = [v / total * 100 for v in values]

fig, ax = plt.subplots(figsize=(8,6), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

for i, v in enumerate(values):
    ax.bar(labels[i], v, color=colors[i], alpha=0.08, width=0.6)
    ax.bar(labels[i], v, color=colors[i], alpha=0.15, width=0.5)

bars = ax.bar(labels, values, color=colors, width=0.4)

for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height,
        f"{values[i]:,}\n({percentages[i]:.2f}%)",
        ha='center',
        va='bottom',
        color="white",
        fontsize=11,
        weight="bold"
    )

ax.set_title(
    "Distribution des Classes — Transactions vs Fraudes",
    fontsize=15,
    color="white",
    pad=20,
    weight="bold"
)

ax.set_xlabel("Classe", color="#C9D1D9")
ax.set_ylabel("Nombre de transactions", color="#C9D1D9")

ax.tick_params(colors="#C9D1D9")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.grid(axis='y', linestyle="--", alpha=0.08)

plt.tight_layout()
plt.show()

### QUESTION 1
**Analysez les résultats ci-dessus et répondez :**

1. Quel est le ratio fraudes/légitimes ? Est-ce un déséquilibre important ?
2. Pourquoi ce déséquilibre est-il un problème pour le Machine Learning ?
3. Quelles techniques pouvez-vous utiliser pour gérer ce déséquilibre ?

**Vos réponses :**

```
1. 0.173% de fraudes.

2. Le modèle peut ignorer la fraude ou le modèle peut avoir une mauvaise detection de la classe fraude.

3. mettre à niveau les données (Oversampling ou undersampling) ou rajouté des poids par classe dans le modèle.
```

### 1.3 Analyse des Distributions

In [ ]:
# TODO: Comparer les statistiques (mean, std, min, max) pour Amount et Time
# entre fraudes et transactions légitimes


legit_count = df[df['Class'] == 0]
fraud_count = df[df['Class'] == 1]

print("="*60)
print("TRANSACTIONS LÉGITIMES")
print("="*60)
print(legit_count[['Amount', 'Time']].agg(['mean', 'std', 'min', 'max']))

print("\n" + "="*60)
print("FRAUDES")
print("="*60)
print(fraud_count[['Amount', 'Time']].agg(['mean', 'std', 'min', 'max']))


In [ ]:
# TODO: Créer 2 histogrammes côte à côte pour visualiser la distribution de Amount
# Un pour les légitimes, un pour les fraudes

fig, ax = plt.subplots(figsize=(10,6), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

counts, bins, patches = ax.hist(
    legit_count['Amount'],
    bins=50,
    alpha=0.7
)

for p in patches:
    p.set_facecolor("#00F5A0")
    p.set_edgecolor("#00F5A0")
    p.set_alpha(0.5)

ax.set_title(
    "Distribution des Montants — Transactions Légitimes",
    fontsize=15,
    color="white",
    pad=20,
    weight="bold"
)

ax.set_xlabel("Montant", color="#C9D1D9")
ax.set_ylabel("Fréquence", color="#C9D1D9")

ax.tick_params(colors="#C9D1D9")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.set_yscale("log")
ax.grid(True, linestyle="--", alpha=0.08)

plt.tight_layout()
plt.show()


# ===============================


fig, ax = plt.subplots(figsize=(10,6), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

counts, bins, patches = ax.hist(
    fraud_count['Amount'],
    bins=50,
    alpha=0.7
)

for p in patches:
    p.set_facecolor("#FF4D6D")
    p.set_edgecolor("#FF4D6D")
    p.set_alpha(0.5)

ax.set_title(
    "Distribution des Montants — Fraudes",
    fontsize=15,
    color="white",
    pad=20,
    weight="bold"
)

ax.set_xlabel("Montant", color="#C9D1D9")
ax.set_ylabel("Fréquence", color="#C9D1D9")

ax.tick_params(colors="#C9D1D9")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.set_yscale("log")
ax.grid(True, linestyle="--", alpha=0.08)

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Créer des boxplots pour comparer Amount entre fraudes et légitimes

fig, ax = plt.subplots(figsize=(9,6), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

data = [legit_count['Amount'], fraud_count['Amount']]

box = ax.boxplot(
    data,
    patch_artist=True,
    labels=['Légitime', 'Fraude'],
    showfliers=True,
    medianprops=dict(color="white", linewidth=2),
    whiskerprops=dict(color="#C9D1D9"),
    capprops=dict(color="#C9D1D9")
)

colors = ["#00F5A0", "#FF4D6D"]

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
    patch.set_edgecolor(color)

for flier, color in zip(box['fliers'], colors):
    flier.set(marker='o',
              markerfacecolor=color,
              markeredgecolor=color,
              alpha=0.3)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.tick_params(colors="#C9D1D9")

ax.set_title(
    "Distribution des Montants — Légitime vs Fraude (échelle log)",
    fontsize=15,
    color="white",
    pad=20,
    weight="bold"
)

ax.set_ylabel("Montant (log scale)", color="#C9D1D9")
ax.set_yscale('log')

ax.grid(True, linestyle="--", alpha=0.08)

plt.tight_layout()
plt.show()

### 1.4 Analyse de Corrélation

In [ ]:
# TODO: Calculer la corrélation de toutes les features avec 'Class'
# Afficher les 10 features les plus corrélées

correlations = df.corr()['Class']

# Sélection des 10 features les plus corrélées
top_10 = correlations.sort_values(ascending=False).head(10)
top_10_corr = top_10.index

# Affichage
print("Top 10 Features corrélées avec Class:")
print(top_10)

In [ ]:
# TODO: Créer une heatmap des 10 features les plus corrélées avec Class

corr_matrix = df[top_10_corr].corr()

plt.figure(figsize=(11, 9), facecolor="#0E1117")
ax = plt.gca()
ax.set_facecolor("#0E1117")

heatmap = sns.heatmap(
    corr_matrix,
    annot=True,
    fmt=".2f",
    cmap="icefire",
    center=0,
    linewidths=0.5,
    linecolor="#1C1F26",
    cbar_kws={"shrink": 0.8}
)

plt.title(
    "Heatmap — Top 10 Features Corrélées avec 'Class'",
    fontsize=16,
    color="white",
    pad=20,
    weight="bold"
)

ax.tick_params(colors="#C9D1D9")
plt.xticks(rotation=45, ha="right", color="#C9D1D9")
plt.yticks(color="#C9D1D9")

cbar = heatmap.collections[0].colorbar
cbar.ax.yaxis.set_tick_params(color="#C9D1D9")
plt.setp(cbar.ax.get_yticklabels(), color="#C9D1D9")

plt.tight_layout()
plt.show()

### QUESTION 2
**Analysez les corrélations :**

1. Quelles sont les 3 features PCA les plus corrélées avec Class ?
2. Les fraudes ont-elles des montants typiques différents des transactions légitimes ?
3. Y a-t-il des patterns temporels visibles ?

**Vos réponses :**

```
1. Les 3 features les plus corrélées avec class sont : V11(0,15), V4(0,13) et V2(0,09)

2. Les fraudes ont globalement des montants plus faible que les transactions légitimes

3. Non
```

### 1.4bis Analyse Temporelle Approfondie

In [ ]:
# TODO: Analyser la distribution des fraudes par heure
# Créer un graphique montrant le nombre de fraudes par heure
# Comparer avec les transactions légitimes


df['Hour'] = (df['Time'] // 3600) % 24

transactions_per_hour = df.groupby('Hour').size()

fraud_df = df[df['Class'] == 1]
legit_df = df[df['Class'] == 0]

fraud_by_hour = fraud_df.groupby('Hour').size()
legit_by_hour = legit_df.groupby('Hour').size()

fig, ax = plt.subplots(figsize=(14,7), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

ax.axvspan(0, 6, color='#1C1F26', alpha=0.4)
ax.axvspan(6, 18, color='#F5F7FA', alpha=0.05)
ax.axvspan(18, 24, color='#1C1F26', alpha=0.4)

x_legit = legit_by_hour.index
y_legit = legit_by_hour.values

for lw, alpha in [(10,0.03),(7,0.06),(5,0.12)]:
    ax.plot(x_legit, y_legit, color="#00F5A0", linewidth=lw, alpha=alpha)

ax.plot(x_legit, y_legit,
        marker='o',
        markersize=6,
        linewidth=2.5,
        color="#00F5A0",
        label="Transactions légitimes")

x_fraud = fraud_by_hour.index
y_fraud = fraud_by_hour.values

for lw, alpha in [(10,0.03),(7,0.06),(5,0.12)]:
    ax.plot(x_fraud, y_fraud, color="#FF4D6D", linewidth=lw, alpha=alpha)

ax.plot(x_fraud, y_fraud,
        marker='o',
        markersize=6,
        linewidth=2.5,
        color="#FF4D6D",
        label="Fraudes")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.tick_params(colors="#C9D1D9")
ax.set_xticks(range(0,24))

ax.set_xlabel("Heure de la journée", color="#C9D1D9", fontsize=12)
ax.set_ylabel("Nombre de transactions", color="#C9D1D9", fontsize=12)

ax.set_title("Transactions légitimes vs Fraudes — Analyse Horaire",
             fontsize=16, color="white", pad=20, weight="bold")

ax.grid(True, linestyle="--", alpha=0.08)

leg = ax.legend(facecolor="#1C1F26", edgecolor="none")
for text in leg.get_texts():
    text.set_color("white")

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Analyser les patterns jour/nuit
# Calculer le taux de fraude pour chaque heure
# Formule: taux_fraude(h) = nb_fraudes(h) / nb_total_transactions(h)
# Identifier les heures à risque élevé

fraud_rate_per_hour = (fraud_by_hour / transactions_per_hour).fillna(0)

seuil = fraud_rate_per_hour.mean() + fraud_rate_per_hour.std()
high_risk_hours = fraud_rate_per_hour[fraud_rate_per_hour > seuil]

fig, ax = plt.subplots(figsize=(14,7), facecolor="#0E1117")
ax.set_facecolor("#0E1117")

ax.axvspan(0, 6, color='#1C1F26', alpha=0.4)
ax.axvspan(6, 18, color='#F5F7FA', alpha=0.05)
ax.axvspan(18, 24, color='#1C1F26', alpha=0.4)

x = fraud_rate_per_hour.index
y = fraud_rate_per_hour.values

for lw, alpha in [(8,0.05),(6,0.08),(4,0.15)]:
    ax.plot(x, y, color="#FF4D6D", linewidth=lw, alpha=alpha)

ax.plot(x, y, marker='o', markersize=6,
        color="#FF4D6D", linewidth=2.5,
        label="Taux de fraude")

ax.fill_between(x, y, seuil, where=(y > seuil),
                color="#FF4D6D", alpha=0.25)

ax.scatter(high_risk_hours.index,
           high_risk_hours.values,
           color="#FFD60A",
           s=120,
           zorder=5,
           label="Heure à risque")

ax.axhline(seuil, color="#00F5D4",
           linestyle="--", linewidth=1.5,
           label="Seuil risque élevé")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.tick_params(colors="#C9D1D9")
ax.set_xticks(range(0,24))

ax.set_xlabel("Heure de la journée", color="#C9D1D9", fontsize=12)
ax.set_ylabel("Taux de fraude", color="#C9D1D9", fontsize=12)
ax.set_title("Analyse Jour / Nuit — Détection des Heures à Risque",
             fontsize=16, color="white", pad=20, weight="bold")

ax.grid(True, linestyle="--", alpha=0.1)

ax.legend(facecolor="#1C1F26", edgecolor="none", labelcolor="white")

plt.tight_layout()
plt.show()

print("Heures à risque élevé :", list(high_risk_hours.index))
print("Taux de fraude élevé :", list(high_risk_hours.values))

In [ ]:
# TODO: Créer une matrice de corrélation entre Time, Amount et les top 5 features PCA
# Visualiser avec une heatmap

top5_pca_cols = ['V1','V2','V3','V4','V5']
df_corr = df[['Time', 'Amount'] + top5_pca_cols]

corr_matrix = df_corr.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(11,9), facecolor="#0E1117")
ax = plt.gca()
ax.set_facecolor("#0E1117")

heatmap = sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt=".2f",
    cmap="icefire",
    center=0,
    linewidths=0.5,
    linecolor="#1C1F26",
    cbar_kws={"shrink": 0.8}
)

plt.title(
    "Matrice de Corrélation — Time, Amount & Top 5 PCA (V1–V5)",
    fontsize=16,
    color="white",
    pad=20,
    weight="bold"
)


ax.tick_params(colors="#C9D1D9")
plt.xticks(rotation=45, ha="right", color="#C9D1D9")
plt.yticks(color="#C9D1D9")

cbar = heatmap.collections[0].colorbar
cbar.ax.yaxis.set_tick_params(color="#C9D1D9")
plt.setp(cbar.ax.get_yticklabels(), color="#C9D1D9")

plt.tight_layout()
plt.show()

---
### 1.5 Feature Engineering Avancé
**Mission : Créer au minimum 15 nouvelles features pertinentes**

In [ ]:
# Créer une copie pour le feature engineering
df_fe = df.copy()

print(f"Shape initiale: {df_fe.shape}")
df_fe.head()

#### Features Temporelles

In [ ]:
# TODO: Créer les features temporelles suivantes :
# 1. 'hour' : Heure de la journée (0-23) à partir de Time
# 2. 'day' : Jour (0 ou 1) à partir de Time
# 3. 'hour_sin' : Encodage cyclique sin de l'heure
# 4. 'hour_cos' : Encodage cyclique cos de l'heure

df_fe['hour'] = (df_fe['Time'] // 3600) % 24
df_fe['day'] = (df_fe['Time'] // (24*3600)) % 2
df_fe['hour_sin'] = np.sin(2 * np.pi * df_fe['hour'] / 24)
df_fe['hour_cos'] = np.cos(2 * np.pi * df_fe['hour'] / 24)

print(" Features temporelles créées")
df_fe[['Time', 'hour', 'day', 'hour_sin', 'hour_cos']].head()

df_fe.head()

In [ ]:
# TODO: Créer une feature 'period' (période de la journée)
# Nuit: 0-6h, Matin: 6-12h, Après-midi: 12-18h, Soir: 18-24h

def get_period(hour):
    """Retourne la période de la journée"""
    if 0 <= hour < 6:
        return 'Nuit'
    elif 6 <= hour < 12:
        return 'Matin'
    elif 12 <= hour < 18:
        return 'Après-midi'
    elif 18 <= hour < 24:
        return 'Soir'
    else:
        return np.nan

df_fe['period'] = df_fe['hour'].apply(get_period)

print("Distribution des périodes:")
print(df_fe['period'].value_counts())

#### Features sur les Montants

In [ ]:
# TODO: Créer les features suivantes sur Amount :
# 1. 'amount_log' : Log transformation (gérer l'asymétrie)
# 2. 'amount_sqrt' : Racine carrée
# 3. 'is_zero_amount' : Indicateur si montant = 0

df_fe['amount_log'] = np.log1p(df_fe['Amount'])
df_fe['amount_sqrt'] = np.sqrt(df_fe['Amount'])
df_fe['is_zero_amount'] = (df_fe['Amount'] == 0).astype(int)

print("Features sur montants créées")
df_fe[['Amount', 'amount_log', 'amount_sqrt', 'is_zero_amount']].head()

In [ ]:
# TODO: Créer un binning du montant
# Catégories suggérées: [0, 10, 50, 100, 500, inf]
# Labels: ['micro', 'small', 'medium', 'large', 'xlarge']

bins = [0, 10, 50, 100, 500, np.inf]
labels = ['micro', 'small', 'medium', 'large', 'xlarge']

df_fe['amount_bin'] = pd.cut(df_fe['Amount'], bins=bins, labels=labels, right=False)

print("Distribution des bins:")
print(df_fe['amount_bin'].value_counts())

#### Features d'Interaction

In [ ]:
# TODO: Créer des features d'interaction
# Multiplier Amount avec les 3 features PCA les plus corrélées avec Class
# (identifiées dans l'analyse de corrélation)
# Exemple: df_fe['V1_amount'] = df_fe['V1'] * df_fe['Amount']

df_fe['V2_amount'] = df_fe['V2'] * df_fe['Amount']
df_fe['V4_amount'] = df_fe['V4'] * df_fe['Amount']
df_fe['V7_amount'] = df_fe['V7'] * df_fe['Amount']

print("Features d'interaction créées")
df_fe[['Amount', 'V2', 'V2_amount', 'V4', 'V4_amount', 'V7', 'V7_amount']].head()

In [ ]:
# TODO: Créer des features d'agrégation sur les V1-V28
# Exemples:
# - Somme des 5 premières features PCA
# - Moyenne des 5 premières features PCA
# - Écart-type des features PCA

pca_cols = [f'V{i}' for i in range(1, 29)]

df_fe['V1_5_sum'] = df_fe[pca_cols[:5]].sum(axis=1)

df_fe['V1_5_mean'] = df_fe[pca_cols[:5]].mean(axis=1)

df_fe['V1_28_std'] = df_fe[pca_cols].std(axis=1)

df_fe['V1_28_median'] = df_fe[pca_cols].median(axis=1)

print("Features d'agrégation créées")
df_fe[['V1_5_sum', 'V1_5_mean', 'V1_28_std', 'V1_28_median']].head()

#### Features Polynomiales et Ratios

In [ ]:
# TODO: Créer des features polynomiales
# 1. amount_squared = Amount²
# 2. amount_cubed = Amount³
# Justification: Capturer les relations non-linéaires

df_fe['amount_squared'] = df_fe['Amount'] ** 2
df_fe['amount_cubed'] = df_fe['Amount'] ** 3

print("Features polynomiales créées")
df_fe[['Amount', 'amount_squared', 'amount_cubed']].head()

In [ ]:
# TODO: Créer des features de ratio
# 1. amount_per_hour = Amount / (hour + 1) pour éviter division par 0
# 2. time_amount_ratio = Time / (Amount + 1)
# Justification: Relations entre variables temporelles et montants

df_fe['amount_per_hour'] = df_fe['Amount'] / (df_fe['hour'] + 1)
df_fe['time_amount_ratio'] = df_fe['Time'] / (df_fe['Amount'] + 1)

print("Features de ratio créées")
df_fe[['Amount', 'hour', 'amount_per_hour', 'Time', 'time_amount_ratio']].head()

In [ ]:
# TODO: Créer des features d'écart à la moyenne
# Pour les 3 features PCA les plus corrélées, calculer:
# deviation_Vi = |Vi - mean(Vi)| / std(Vi)
# Justification: Identifier les valeurs anormales (z-score absolu)

top3_pca = ['V11', 'V4', 'V2']

for col in top3_pca:
    mean_val = df_fe[col].mean()
    std_val = df_fe[col].std()
    df_fe[f'deviation_{col}'] = np.abs(df_fe[col] - mean_val) / std_val

print("Features d'écart créées")
df_fe[[f'deviation_{col}' for col in top3_pca]].head()

#### Validation des Nouvelles Features

In [ ]:
# Compter les nouvelles features créées
original_features = df.shape[1]
new_features_count = df_fe.shape[1] - original_features

print(f"Features originales: {original_features}")
print(f"Nouvelles features créées: {new_features_count}")
print(f"Total features: {df_fe.shape[1]}")

# Lister les nouvelles features
new_features = [col for col in df_fe.columns if col not in df.columns]
print(f"\nNouvelles features: {new_features}")

In [ ]:
# TODO: Analyser la corrélation des nouvelles features avec Class
# Afficher les nouvelles features triées par corrélation absolue

original_cols = [f'V{i}' for i in range(1,29)] + ['Time','Amount','Class']
new_features = [col for col in df_fe.columns if col not in original_cols and col not in ['period', 'amount_bin']]

corr_with_class = df_fe[new_features + ['Class']].corr()['Class'].drop('Class')

corr_sorted = corr_with_class.reindex(corr_with_class.abs().sort_values(ascending=False).index)

print("Corrélation des nouvelles features avec Class :")
print(corr_sorted)

### QUESTION 3
**Justifiez vos choix de Feature Engineering :**

1. Quelles features créées sont les plus prometteuses (corrélation) ?
2. Pourquoi l'encodage cyclique (sin/cos) est-il pertinent pour l'heure ?
3. Quel est l'intérêt métier de créer des interactions Amount × V_i ?

**Vos réponses :**

```
1. V1_28_std, deviation_V11, deviation_V4, V1_28_median, deviation_V2

2. pour que 22H soit plus proche de 2H

3. pour detecter les montants élevé combiner aux features PCA inhabituelle, qui normalement sont plus suspect
```

---
## PARTIE 2 : MODÉLISATION BASELINE (1h30)
---

### 2.1 Préparation des Données

In [ ]:
# TODO: Séparer X et y
# Attention: Exclure les colonnes catégorielles non encodées (ex: 'period', 'amount_bin')
# ou les encoder avant avec pd.get_dummies()

# Encoder les variables catégorielles si nécessaire
# df_fe_encoded = pd.get_dummies(df_fe, columns=['period', 'amount_bin'], drop_first=True)

exclude_cols = ['Class', 'period', 'amount_bin']
X = df_fe.drop(columns=exclude_cols)
y = df_fe['Class']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

In [ ]:
# TODO: Split Train/Test
# Utiliser stratify=y pour préserver le ratio des classes
# Test size: 20%

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

print(f"\nDistribution dans le train:")
print(y_train.value_counts())

print(f"\nDistribution dans le test:")
print(y_test.value_counts())

In [ ]:
# TODO: Scaling
# Choisir entre StandardScaler ou RobustScaler
# RobustScaler est recommandé car il gère mieux les outliers

scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Scaling effectué")

### QUESTION 4
**Justifiez votre choix de scaler :**

```
J'ai choisi [StandardScaler/RobustScaler] parce que Certaines features comme Amount et les agrégations PCA (V1_28_std, V1_28_median, etc.) contiennent des valeurs extrêmes.
```

### 2.2 Modèles Baseline

In [ ]:
# Fonction utilitaire pour évaluer un modèle
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Entraîne et évalue un modèle

    Returns:
        dict: Dictionnaire avec les métriques
    """
    print(f"\n{'='*60}")
    print(f"ÉVALUATION: {model_name}")
    print(f"{'='*60}")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        y_proba = model.decision_function(X_test)
    else:
        y_proba = y_pred

    cm = confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)
    pr_auc = average_precision_score(y_test, y_proba)

    print("Confusion Matrix:")
    print(cm)
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")
    print(f"F1-score  : {f1:.4f}")
    print(f"ROC-AUC   : {roc_auc:.4f}")
    print(f"PR-AUC    : {pr_auc:.4f}")

    metrics = {
        "confusion_matrix": cm,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc
    }

    return metrics

print("Fonction d'évaluation créée")

#### Modèle 1 : Logistic Regression

In [ ]:
# TODO: Créer et évaluer une Logistic Regression
# Utiliser class_weight='balanced' pour gérer le déséquilibre

lr_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

# Évaluer le modèle
lr_results = evaluate_model(
    lr_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="Logistic Regression"
)

#### Modèle 2 : Decision Tree

In [ ]:
# TODO: Créer et évaluer un Decision Tree
# Paramètres suggérés: max_depth=10, class_weight='balanced'

dt_model = DecisionTreeClassifier(
    max_depth=10,
    class_weight='balanced',
    random_state=42
)

dt_results = evaluate_model(
    dt_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="Decision Tree"
)

#### Modèle 3 : Random Forest

In [ ]:
# TODO: Créer et évaluer un Random Forest
# Paramètres suggérés: n_estimators=100, class_weight='balanced'

rf_model = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42
)

# Évaluer le modèle
rf_results = evaluate_model(
    rf_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="Random Forest"
)

#### Modèle 4 : Support Vector Machine (SVM)

In [ ]:
# TODO: Créer et évaluer un SVM
# Paramètres: kernel='rbf', class_weight='balanced', C=1.0
# Note: SVM peut être lent sur de grandes données

svm_model = SVC(
    kernel='rbf',
    class_weight='balanced',
    C=1.0,
    random_state=42
)

svm_results = evaluate_model(
    svm_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="SVM"
)

#### Modèle 5 : K-Nearest Neighbors (KNN)

In [ ]:
# TODO: Créer et évaluer un KNN
# Paramètres: n_neighbors=5, weights='distance'
# Note: KNN nécessite des données scalées

from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance',
    n_jobs=-1
)

knn_results = evaluate_model(
    knn_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="KNN"
)

#### Modèle 6 : XGBoost (Baseline)

In [ ]:
# TODO: Créer et évaluer un XGBoost baseline
# Paramètres: n_estimators=100, max_depth=5, learning_rate=0.1
# scale_pos_weight = nb_negatifs / nb_positifs (pour gérer le déséquilibre)

from xgboost import XGBClassifier

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Scale pos weight: {scale_pos_weight:.2f}")

xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

xgb_results = evaluate_model(
    xgb_model,
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    model_name="XGBoost"
)

### 2.3 Comparaison des Modèles Baseline

In [ ]:
# TODO: Créer un tableau comparatif des 6 modèles
# Colonnes: Model, Precision, Recall, F1, ROC-AUC, PR-AUC
# pd.DataFrame([lr_results, dt_results, rf_results, svm_results, knn_results, xgb_results])

results_list = [
    {"Model": "Logistic Regression", **lr_results},
    {"Model": "Decision Tree", **dt_results},
    {"Model": "Random Forest", **rf_results},
    {"Model": "KNN", **knn_results},
    {"Model": "XGBoost", **xgb_results}
]

comparison_df = pd.DataFrame(results_list)

comparison_df = comparison_df[["Model", "precision", "recall", "f1_score", "roc_auc", "pr_auc"]]

print("\n" + "="*60)
print("COMPARAISON DES MODÈLES BASELINE")
print("="*60)
print(comparison_df)

In [ ]:
# TODO: Visualiser la comparaison avec un bar plot
# Afficher Precision, Recall, F1, PR-AUC pour les 3 modèles

top_models = comparison_df[comparison_df['Model'].isin([
    "Logistic Regression", "Random Forest", "XGBoost"
])]

plot_data = top_models.melt(
    id_vars='Model',
    value_vars=['precision', 'recall', 'f1_score', 'pr_auc'],
    var_name='Metric',
    value_name='Score'
)

metric_colors = {
    'precision': "#00F5A0",
    'recall': "#FFD60A",
    'f1_score': "#FF4D6D",
    'pr_auc': "#4CC9F0"
}

plt.figure(figsize=(11,6), facecolor="#0E1117")
ax = plt.gca()
ax.set_facecolor("#0E1117")

sns.barplot(
    x='Model',
    y='Score',
    hue='Metric',
    data=plot_data,
    palette=metric_colors,
    edgecolor="white",
    alpha=0.85
)

plt.ylim(0,1)
plt.title(
    "Comparaison des Métriques — Modèles Baseline",
    fontsize=16,
    color="white",
    pad=20,
    weight="bold"
)

plt.xlabel("Modèle", color="#C9D1D9")
plt.ylabel("Score", color="#C9D1D9")

ax.tick_params(colors="#C9D1D9")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color("#2A2F3A")
ax.spines['bottom'].set_color("#2A2F3A")

ax.grid(axis='y', linestyle="--", alpha=0.08)

leg = ax.legend(title='Métrique', facecolor="#1C1F26", edgecolor="none")
for text in leg.get_texts():
    text.set_color("white")
leg.get_title().set_color("white")

plt.tight_layout()
plt.show()

### 2.4 Comparaison des Stratégies de Rééquilibrage

In [ ]:
# TODO: Comparer class_weight='balanced' vs SMOTE
# Prendre le meilleur modèle baseline et tester avec SMOTE
# Formule SMOTE: Génère des exemples synthétiques pour la classe minoritaire
# Pour chaque exemple minoritaire x_i, choisir k voisins et créer:
# x_new = x_i + λ * (x_neighbor - x_i) où λ ∈ [0,1]

from imblearn.over_sampling import SMOTE


RANDOM_STATE = 42

smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"Avant SMOTE: {y_train.value_counts().to_dict()}")
print(f"Après SMOTE: {pd.Series(y_train_smote).value_counts().to_dict()}")

xgb_smote_model = XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=RANDOM_STATE,
    n_jobs=-1
)


xgb_smote_results = evaluate_model(
    xgb_smote_model,
    X_train_smote,
    X_test_scaled,
    y_train_smote,
    y_test,
    model_name="XGBoost + SMOTE"
)

In [ ]:
# TODO: Comparer les résultats
# Créer un tableau: Stratégie | Precision | Recall | F1 | PR-AUC
# Ligne 1: class_weight='balanced'
# Ligne 2: SMOTE
# Analyser quelle stratégie est la plus efficace

comparison_strategy_df = pd.DataFrame([
    {
        "Stratégie": "Class Weight = 'balanced'",
        "Precision": xgb_results['precision'],
        "Recall": xgb_results['recall'],
        "F1": xgb_results['f1_score'],
        "PR-AUC": xgb_results['pr_auc']
    },
    {
        "Stratégie": "SMOTE",
        "Precision": xgb_smote_results['precision'],
        "Recall": xgb_smote_results['recall'],
        "F1": xgb_smote_results['f1_score'],
        "PR-AUC": xgb_smote_results['pr_auc']
    }
])

print("\n" + "="*60)
print("COMPARAISON DES STRATÉGIES D'ÉQUILIBRAGE")
print("="*60)
print(comparison_strategy_df)

### QUESTION 5
**Analysez les résultats baseline :**

1. Parmi les 6 modèles testés, lequel a le meilleur Recall ? Est-ce suffisant (objectif ≥ 0.85) ?
2. Pourquoi PR-AUC est-il plus pertinent que ROC-AUC ici ?
3. Comparez class_weight vs SMOTE. Quelle stratégie est la plus efficace ?
4. Quel modèle choisiriez-vous pour l'optimisation ? Pourquoi ?

**Vos réponses :**

```
1. le model de Logistic Regression, oui (0,9)

2. car on a des données déséquilibré

3. le plus efficace est le class_weight avec une meilleur Precision et F1

4. le model de Random Forest car il a le meilleur compromis entre Précision (0,9) et Recall (0,75)
```

---
## PARTIE 3 : OPTIMISATION AVANCÉE (2h)
---

**Note** : Cette partie peut prendre du temps (GridSearch = 15-30 min). Commencez par une petite grille pour tester !

### 3.1 Construction d'un Pipeline ML Complet

In [ ]:
# TODO: Créer un Pipeline intégrant:
# 1. Scaling (RobustScaler)
# 2. Modèle (Random Forest - le meilleur baseline)

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('rf', RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

print("Pipeline créé")
print(pipeline)

### 3.2 Optimisation Hyperparamètres - GridSearchCV

In [ ]:
# TODO: Définir une grille de paramètres pour Random Forest
# IMPORTANT: Préfixer les paramètres avec 'classifier__' car dans un pipeline
# Commencez PETIT pour tester (2-3 valeurs par paramètre max)
# Exemple:
# param_grid = {
# 'classifier__n_estimators': [100, 200],
# 'classifier__max_depth': [10, None],
# 'classifier__min_samples_split': [2, 5],
# 'classifier__class_weight': ['balanced']
# }

param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [10, None],
    'rf__min_samples_split': [2, 5],
    'rf__class_weight': ['balanced']
}

n_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"Nombre de combinaisons: {n_combinations}")
print(f"Avec cv=5 → {n_combinations * 5} entraînements")

In [ ]:
# TODO: Configurer GridSearchCV
# - cv: StratifiedKFold(n_splits=5)
# - scoring: 'average_precision' (PR-AUC)
# - n_jobs: -1 (parallélisation)
# - verbose: 2 (afficher la progression)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='average_precision',
    cv=cv,
    n_jobs=-1,
    verbose=2
)

print(" GridSearchCV configuré")

In [ ]:
# TODO: Lancer la recherche (peut prendre plusieurs minutes)
# Utiliser X_train et y_train (NON scalés, le pipeline s'en occupe)

import time
start_time = time.time()

grid_search.fit(X_train, y_train)

end_time = time.time()
print(f"\nTemps d'exécution: {(end_time - start_time) / 60:.2f} minutes")

In [ ]:
# TODO: Afficher les résultats de GridSearch
# - Meilleurs paramètres (grid_search.best_params_)
# - Meilleur score CV (grid_search.best_score_)
# - Évaluer sur le test set (grid_search.best_estimator_)

print("="*60)
print("RÉSULTATS GRIDSEARCH")
print("="*60)

print("Meilleurs paramètres trouvés :")
print(grid_search.best_params_)

print(f"\nMeilleur score cross-validation (PR-AUC) : {grid_search.best_score_:.4f}")

best_model = grid_search.best_estimator_
best_results = evaluate_model(
    best_model,
    X_train,
    X_test,
    y_train,
    y_test,
    model_name="Random Forest Optimisé"
)

### 3.3 Optimisation avec RandomizedSearchCV

In [ ]:
# TODO: Comparer GridSearchCV vs RandomizedSearchCV
# RandomizedSearchCV teste n_iter combinaisons aléatoires au lieu de toutes
# Avantage: Plus rapide, peut explorer un espace plus large
# Formule: Probabilité de trouver le top 5% en testant n combinaisons aléatoires:
# P = 1 - (0.95)^n
# Exemple: n=20 → P ≈ 64%, n=60 → P ≈ 95%

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Définir des distributions de paramètres (plus larges que GridSearch)
param_distributions = {
    'rf__n_estimators': randint(50, 300),
    'rf__max_depth': [5, 10, 15, 20, 25, None],
    'rf__min_samples_split': randint(2, 20),
    'rf__min_samples_leaf': randint(1, 10),
    'rf__max_features': ['sqrt', 'log2', None],
    'rf__class_weight': ['balanced']
}

# Configurer RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions,
    n_iter=30,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring='average_precision',
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=2
)

print("RandomizedSearchCV configuré")

In [ ]:
# TODO: Lancer RandomizedSearchCV
import time
start_time = time.time()


random_search.fit(X_train, y_train)

end_time = time.time()
print(f"\nTemps d'exécution: {(end_time - start_time) / 60:.2f} minutes")

print("\n" + "="*60)
print("RÉSULTATS RANDOMIZEDSEARCH")
print("="*60)
print(f"Meilleurs paramètres: {random_search.best_params_}")
print(f"Meilleur score (CV - PR-AUC): {random_search.best_score_:.4f}")

In [ ]:
# TODO: Comparer GridSearch vs RandomizedSearch
# Créer un tableau comparatif:
# Méthode | Meilleur Score | Temps (min) | Nb Combinaisons Testées

comparaison_search = pd.DataFrame({
    'Méthode': ['GridSearchCV', 'RandomizedSearchCV'],
    'Meilleur_Score': [grid_score, random_score],
    'Temps_min': [grid_time_min, random_time_min],
    'Nb_Combinaisons': [grid_combinations, random_combinations]
})

print("\n" + "="*60)
print("COMPARAISON DES MÉTHODES D'OPTIMISATION")
print("="*60)
print(comparaison_search)

### 3.4 Optimisation de XGBoost

In [ ]:
# TODO: Optimiser XGBoost avec RandomizedSearchCV
# Créer un pipeline avec XGBoost
# Paramètres à optimiser:
# - n_estimators: [50, 100, 200, 300]
# - max_depth: [3, 5, 7, 9]
# - learning_rate: [0.01, 0.05, 0.1, 0.3]
# - subsample: [0.6, 0.8, 1.0]
# - colsample_bytree: [0.6, 0.8, 1.0]
# - scale_pos_weight: calculé automatiquement

pipeline_xgb = Pipeline([
    ('scaler', RobustScaler()),
    ('classifier', XGBClassifier(
        random_state=42, 
        eval_metric='logloss',
        use_label_encoder=False,
        scale_pos_weight=scale_pos_weight
    ))
])

param_distributions = {
    'classifier__n_estimators': [50, 100, 200, 300],
    'classifier__max_depth': [3, 5, 7, 9],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.3],
    'classifier__subsample': [0.6, 0.8, 1.0],
    'classifier__colsample_bytree': [0.6, 0.8, 1.0]
}

random_search_xgb = RandomizedSearchCV(
    pipeline_xgb,
    param_distributions=param_distributions,
    n_iter=30,
    scoring='roc_auc',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search_xgb.fit(X_train, y_train)

print("Meilleurs paramètres :", random_search_xgb.best_params_)
print("Meilleur score ROC-AUC :", random_search_xgb.best_score_)


### QUESTION 6
**Analysez l'optimisation :**

1. Quels hyperparamètres ont le plus d'impact sur la performance ?
2. Le gain de performance justifie-t-il le temps de calcul ?
3. GridSearch vs RandomizedSearch: quelle méthode est la plus efficace ?
4. XGBoost optimisé vs Random Forest optimisé: lequel est meilleur ?
5. Atteignez-vous l'objectif de Recall ≥ 0.85 ?

---
## PARTIE 4 : ANALYSE & DIAGNOSTIC (1h)
---

### 4.1 Feature Importance

In [ ]:
# TODO: Extraire l'importance des features du meilleur modèle

feature_importances = best_model.feature_importances_

df_features = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

df_features = df_features.sort_values(by='Importance', ascending=False)

top_15_features = df_features.head(15)
print(top_15_features)

In [ ]:
# TODO: Visualiser le Top 15 avec un bar plot horizontal

plt.figure(figsize=(10, 6))
sns.barplot(
    x='Importance', 
    y='Feature', 
    data=top_15_features,
    palette='viridis'
)

plt.title("Top 15 des features les plus importantes")
plt.xlabel("Importance")
plt.ylabel("Feature")

plt.tight_layout()
plt.show()

### 4.1bis Permutation Importance

In [ ]:
# TODO: Calculer la Permutation Importance
# Principe: Mélanger aléatoirement une feature et mesurer la baisse de performance
# Formule: PI(f) = Score_original - Score_après_permutation(f)
# Plus PI est élevé, plus la feature est importante

from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(
    best_model,
    X_test,
    y_test,
    n_repeats=10,
    random_state=RANDOM_STATE,
    scoring='average_precision'
)

perm_imp_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance_Mean': perm_importance.importances_mean,
    'Importance_Std': perm_importance.importances_std
}).sort_values('Importance_Mean', ascending=False)

top15_perm = perm_imp_df.head(15)
print("Top 15 Features - Permutation Importance:")
print(top15_perm)

plt.figure(figsize=(10, 6))
sns.barplot(
    x='Importance_Mean',
    y='Feature',
    data=top15_perm,
    palette='magma',
    xerr=top15_perm['Importance_Std']
)
plt.title("Top 15 Features par Permutation Importance")
plt.xlabel("Permutation Importance (mean ± std)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Comparer MDI (Mean Decrease Impurity) vs Permutation Importance
# Créer un graphique côte à côte
# MDI peut être biaisé vers les features à haute cardinalité
# Permutation Importance est plus fiable mais plus coûteux en calcul

mdi_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_model.feature_importances_
}).sort_values('Importance', ascending=False).head(15)
mdi_df = mdi_df[::-1]

perm_df = perm_imp_df.head(15)
perm_df = perm_df[::-1]

fig, axes = plt.subplots(ncols=2, figsize=(14, 8), sharey=True)

sns.barplot(
    x='Importance',
    y='Feature',
    data=mdi_df,
    ax=axes[0],
    palette='viridis'
)
axes[0].set_title("Top 15 Features - MDI (Mean Decrease Impurity)")
axes[0].set_xlabel("Importance")
axes[0].set_ylabel("Feature")

sns.barplot(
    x='Importance_Mean',
    y='Feature',
    data=perm_df,
    ax=axes[1],
    palette='magma',
    xerr=perm_df['Importance_Std']
)
axes[1].set_title("Top 15 Features - Permutation Importance")
axes[1].set_xlabel("Importance (mean ± std)")
axes[1].set_ylabel("")

plt.tight_layout()
plt.show()

### 4.1ter SHAP Values (Explicabilité Avancée)

In [ ]:
# TODO: Calculer les SHAP values
# SHAP (SHapley Additive exPlanations) basé sur la théorie des jeux
# Formule de Shapley: φ_i = Σ [|S|!(n-|S|-1)!/n!] × [f(S∪{i}) - f(S)]
# où S sont tous les sous-ensembles de features sans i
# Interprétation: Contribution marginale moyenne de chaque feature

import shap

explainer = shap.TreeExplainer(best_model.named_steps['classifier'])

X_test_sample = X_test.iloc[:100]

# 3️⃣ Calculer les SHAP values
shap_values = explainer.shap_values(X_test_sample)

if isinstance(shap_values, list):
    print(f"SHAP values shape (class 1): {shap_values[1].shape}")
else:
    print(f"SHAP values shape: {shap_values.shape}")

shap.summary_plot(shap_values[1] if isinstance(shap_values, list) else shap_values,
                  X_test_sample,
                  plot_type="bar")

In [ ]:
# TODO: Visualiser le summary plot SHAP
# Ce graphique montre:
# - L'importance de chaque feature (axe Y)
# - L'impact sur la prédiction (axe X)
# - La valeur de la feature (couleur: rouge=élevé, bleu=faible)

shap_values_plot = shap_values[1] if isinstance(shap_values, list) else shap_values

shap.summary_plot(shap_values_plot, X_test_sample, plot_type="dot", show=False)
plt.tight_layout()
plt.show()

In [ ]:
# TODO: Expliquer une prédiction individuelle
# Choisir une fraude correctement détectée et analyser pourquoi
# SHAP force plot montre la contribution de chaque feature à la prédiction

fraud_idx = y_test[y_test == 1].index[0]
fraud_sample_idx = X_test.index.get_loc(fraud_idx)

if fraud_sample_idx < 100: # Si dans notre échantillon
    print(f"Analyse de la transaction {fraud_idx} (Fraude)")
    print(f"Probabilité prédite: {best_model.predict_proba(X_test.iloc[[fraud_sample_idx]])[:,1][0]:.4f}")

    shap.force_plot(
    explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
    shap_values_plot[fraud_sample_idx],
    X_test_sample.iloc[fraud_sample_idx],
    matplotlib=True,
    show=False
)

plt.tight_layout()
plt.show()

### QUESTION 7
**Interprétez les Feature Importances :**

1. Les features créées (Feature Engineering) sont-elles utiles ?
2. Y a-t-il des surprises (features inattendues importantes) ?
3. MDI vs Permutation Importance: quelles différences observez-vous ?
4. SHAP: quelles features contribuent le plus aux prédictions de fraude ?
5. Pourrait-on simplifier le modèle en retirant des features peu importantes ?

**Vos réponses :**

```
1. ...

2. ...

3. ...

4. ...

5. ...
```

### 4.2 Courbes ROC et Precision-Recall

In [ ]:
# TODO: Tracer les courbes ROC et Precision-Recall côte à côte
# 1. Prédire les probabilités avec predict_proba()
# 2. Calculer les courbes avec roc_curve() et precision_recall_curve()
# 3. Visualiser avec plt.subplot(1, 2, 1) et plt.subplot(1, 2, 2)

y_proba = best_model.predict_proba(X_test)[:, 1]

fpr, tpr, roc_thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
avg_precision = average_precision_score(y_test, y_proba)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(recall, precision, color='purple', lw=2, label=f'AP = {avg_precision:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')

plt.tight_layout()
plt.show()

### 4.3 Learning Curves

In [ ]:
# TODO: Tracer les Learning Curves
# Utiliser learning_curve de sklearn
# Paramètres: cv=5, scoring='average_precision', train_sizes=np.linspace(0.1, 1.0, 10)

train_sizes, train_scores, test_scores = learning_curve(
    best_model,
    X_train,
    y_train,
    cv=5,
    scoring='average_precision',
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2, color='blue')
plt.plot(train_sizes, test_mean, 'o-', color='green', label='Cross-validation score')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.2, color='green')
plt.xlabel('Training Size')
plt.ylabel('Average Precision')
plt.title('Learning Curves')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


### 4.3bis Calibration des Probabilités

In [ ]:
# TODO: Vérifier la calibration du modèle
# Un modèle bien calibré: si proba=0.8, alors 80% des prédictions sont correctes
# Méthode: Tracer la courbe de calibration (reliability diagram)
# Axe X: Probabilité prédite (bins)
# Axe Y: Fraction réelle de positifs dans chaque bin
# Ligne diagonale = calibration parfaite

from sklearn.calibration import calibration_curve

# Calculer la courbe de calibration
fraction_of_positives, mean_predicted_value = calibration_curve(
 y_test,
 y_proba_optimized,
 n_bins=10,
 strategy='uniform'
)

# Visualiser
plt.figure(figsize=(10, 6))
plt.plot(mean_predicted_value, fraction_of_positives, 's-', label='Modèle')
plt.plot([0, 1], [0, 1], 'k--', label='Calibration parfaite')
plt.xlabel('Probabilité prédite moyenne')
plt.ylabel('Fraction de positifs')
plt.title('Courbe de Calibration')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Interprétation:")
print("- Si proche de la diagonale: modèle bien calibré")
print("- Si au-dessus: modèle sous-estime les probabilités")
print("- Si en-dessous: modèle surestime les probabilités")

In [ ]:
# TODO: Appliquer la calibration si nécessaire
# Méthode: CalibratedClassifierCV avec Platt Scaling ou Isotonic Regression
# Platt Scaling: Ajuste une régression logistique sur les probabilités
# Formule: P_calibrated = 1 / (1 + exp(A × log(p/(1-p)) + B))
# Isotonic Regression: Ajuste une fonction monotone non-paramétrique

from sklearn.calibration import CalibratedClassifierCV

# Calibrer le modèle
calibrated_model = CalibratedClassifierCV(
 best_model,
 method='sigmoid', # Platt Scaling
 cv=5
)

# Entraîner sur le train
calibrated_model.fit(X_train, y_train)

# Prédire sur le test
y_proba_calibrated = calibrated_model.predict_proba(X_test)[:, 1]

# Comparer les performances
print("\nComparaison avant/après calibration:")
print(f"PR-AUC avant: {average_precision_score(y_test, y_proba_optimized):.4f}")
print(f"PR-AUC après: {average_precision_score(y_test, y_proba_calibrated):.4f}")

### QUESTION 8
**Diagnostiquez le modèle :**

1. Le modèle est-il en overfitting, underfitting ou bon fit ?
2. Le modèle bénéficierait-il de plus de données ?
3. Quelles actions recommanderiez-vous pour améliorer le modèle ?

### 4.4 Optimisation du Seuil de Décision

In [ ]:
# TODO: Trouver le seuil optimal maximisant le F1-Score
# 1. Calculer precision/recall pour tous les seuils avec precision_recall_curve()
# 2. Calculer F1 pour chaque seuil: F1 = 2 * (P * R) / (P + R)
# 3. Trouver le seuil qui maximise F1
# 4. Comparer les métriques avec seuil 0.5 vs seuil optimal

precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

y_pred_default = (y_proba >= 0.5).astype(int)
y_pred_optimal = (y_proba >= optimal_threshold).astype(int)

from sklearn.metrics import precision_score, recall_score, f1_score

metrics_default = {
    "Precision": precision_score(y_test, y_pred_default),
    "Recall": recall_score(y_test, y_pred_default),
    "F1": f1_score(y_test, y_pred_default)
}

metrics_optimal = {
    "Precision": precision_score(y_test, y_pred_optimal),
    "Recall": recall_score(y_test, y_pred_optimal),
    "F1": f1_score(y_test, y_pred_optimal)
}

print(f"Seuil optimal: {optimal_threshold:.3f}")
print("Metrics seuil 0.5:", metrics_default)
print("Metrics seuil optimal:", metrics_optimal)


### QUESTION 9
**Analysez l'optimisation du seuil :**

1. Quel est l'impact sur Precision et Recall ?
2. Quel seuil recommanderiez-vous en production ? Pourquoi ?
3. Comment choisir entre privilégier Precision ou Recall selon le contexte métier ?

---
## PARTIE 5 : DÉPLOIEMENT & PRODUCTION (1h)
---

### 5.1 Validation Temporelle (TimeSeriesSplit)

In [ ]:
# TODO: Valider le modèle avec TimeSeriesSplit
# Comparer avec StratifiedKFold

from sklearn.model_selection import TimeSeriesSplit, cross_val_score, StratifiedKFold

tscv = TimeSeriesSplit(n_splits=5)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

scores_ts = cross_val_score(best_model, X_train, y_train, cv=tscv, scoring='average_precision', n_jobs=-1)
scores_skf = cross_val_score(best_model, X_train, y_train, cv=skf, scoring='average_precision', n_jobs=-1)

print("TimeSeriesSplit Average Precision:", scores_ts, "Mean:", scores_ts.mean())
print("StratifiedKFold Average Precision:", scores_skf, "Mean:", scores_skf.mean())


### 5.2 Sérialisation du Modèle

In [ ]:
# TODO: Sauvegarder le meilleur modèle (pipeline complet)

import joblib
import os

model_path = '../models/best_model.pkl'
joblib.dump(best_model, model_path)
print(f"Modèle sauvegardé dans : {model_path}")

In [ ]:
# TODO: Sauvegarder les métadonnées
# Inclure: version, date, paramètres, scores, seuil optimal, features, etc.

metadata.update({
    'model_params': best_model.get_params(),
    'average_precision_train': cross_val_score(best_model, X_train, y_train, cv=5, scoring='average_precision').mean(),
    'average_precision_test': average_precision_score(y_test, y_proba),
    'optimal_threshold': optimal_threshold,
    'feature_names': list(X_train.columns)
})

metadata_path = '../models/best_model_metadata.pkl'
joblib.dump(metadata, metadata_path)
print(f"Métadonnées sauvegardées dans : {metadata_path}")


### 5.3 Test de Chargement et Prédiction

In [ ]:
# TODO: Charger le modèle sauvegardé et tester une prédiction
# 1. Charger avec joblib.load()
# 2. Prédire sur un échantillon du test
# 3. Afficher les résultats

loaded_model = joblib.load('../models/best_model.pkl')

X_sample = X_test.iloc[:5]
y_sample_true = y_test.iloc[:5]

y_sample_proba = loaded_model.predict_proba(X_sample)[:, 1]
y_sample_pred = (y_sample_proba >= optimal_threshold).astype(int)

for i in range(len(X_sample)):
    print(f"Échantillon {i+1}: Probabilité={y_sample_proba[i]:.3f}, Prédiction={y_sample_pred[i]}, Vérité={y_sample_true.iloc[i]}")


### 5.4 Création d'une API de Prédiction (Flask)

In [ ]:
# TODO: Créer un script Flask pour l'API
# Structure:
# - Endpoint POST /predict
# - Input: JSON avec les features d'une transaction
# - Output: JSON avec {"is_fraud": bool, "probability": float, "risk_level": str}

# Créer le fichier api_fraud_detection.py
api_code = '''
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np

app = Flask(__name__)

# Charger le modèle au démarrage
model = joblib.load('models/fraud_detector_v1.joblib')
metadata = joblib.load('models/metadata_v1.joblib')

@app.route('/predict', methods=['POST'])
def predict():
 """
 Endpoint de prédiction
 Input JSON: {"Time": float, "V1": float, ..., "Amount": float}
 Output JSON: {"is_fraud": bool, "probability": float, "risk_level": str}
 """
 try:
 # Récupérer les données
 data = request.get_json()

 # Convertir en DataFrame
 df = pd.DataFrame([data])

 # Prédire
 proba = model.predict_proba(df)[:, 1][0]
 threshold = metadata.get('optimal_threshold', 0.5)
 is_fraud = proba >= threshold

 # Déterminer le niveau de risque
 if proba < 0.3:
 risk_level = 'low'
 elif proba < 0.6:
 risk_level = 'medium'
 elif proba < 0.85:
 risk_level = 'high'
 else:
 risk_level = 'critical'

 # Retourner la réponse
 return jsonify({
 'is_fraud': bool(is_fraud),
 'probability': float(proba),
 'risk_level': risk_level,
 'threshold_used': float(threshold)
 })

 except Exception as e:
 return jsonify({'error': str(e)}), 400

@app.route('/health', methods=['GET'])
def health():
 """Endpoint de santé"""
 return jsonify({'status': 'ok', 'model_version': metadata.get('model_version', 'unknown')})

if __name__ == '__main__':
 app.run(host='0.0.0.0', port=5000, debug=False)
'''

# Sauvegarder le fichier
with open('../api_fraud_detection.py', 'w') as f:
 f.write(api_code)

print("API Flask créée: ../api_fraud_detection.py")
print("\nPour lancer l'API:")
print(" python api_fraud_detection.py")
print("\nPour tester:")
print(" curl -X POST http://localhost:5000/predict -H 'Content-Type: application/json' -d '{...}'")

### 5.5 Tests Unitaires du Modèle

In [ ]:
# TODO: Créer des tests unitaires pour valider le modèle
# Tests à implémenter:
# 1. Test de chargement du modèle
# 2. Test de prédiction sur des données valides
# 3. Test de robustesse (valeurs manquantes, outliers)
# 4. Test de performance (temps de prédiction < 100ms)

import unittest
import time

class TestFraudDetector(unittest.TestCase):

 @classmethod
 def setUpClass(cls):
 """Charger le modèle une fois pour tous les tests"""
 cls.model = joblib.load('../models/fraud_detector_v1.joblib')
 cls.sample_data = X_test.iloc[:10]

 def test_model_loading(self):
 """Test: Le modèle se charge correctement"""
 self.assertIsNotNone(self.model)
 print("Test chargement: OK")

 def test_prediction_shape(self):
 """Test: Les prédictions ont la bonne forme"""
 predictions = self.model.predict(self.sample_data)
 self.assertEqual(len(predictions), len(self.sample_data))
 print("Test shape prédictions: OK")

 def test_prediction_values(self):
 """Test: Les prédictions sont dans [0, 1]"""
 probas = self.model.predict_proba(self.sample_data)[:, 1]
 self.assertTrue(all(0 <= p <= 1 for p in probas))
 print("Test valeurs prédictions: OK")

 def test_prediction_time(self):
 """Test: Temps de prédiction < 100ms pour 10 transactions"""
 start = time.time()
 _ = self.model.predict(self.sample_data)
 elapsed = (time.time() - start) * 1000 # en ms
 self.assertLess(elapsed, 100)
 print(f"Test temps prédiction: {elapsed:.2f}ms < 100ms OK")

# Exécuter les tests
if __name__ == '__main__':
 suite = unittest.TestLoader().loadTestsFromTestCase(TestFraudDetector)
 runner = unittest.TextTestRunner(verbosity=2)
 result = runner.run(suite)

 print(f"\nTests exécutés: {result.testsRun}")
 print(f"Succès: {result.testsRun - len(result.failures) - len(result.errors)}")
 print(f"Échecs: {len(result.failures)}")
 print(f"Erreurs: {len(result.errors)}")

---
## SYNTHÈSE FINALE
---

### QUESTION 10 - BILAN GLOBAL
**Rédigez une synthèse de votre travail (10-15 lignes) :**

1. **Performance finale** : Avez-vous atteint les objectifs (Recall ≥ 0.85, Precision maximale) ?
2. **Feature Engineering** : Quelles features créées ont été les plus utiles ?
3. **Optimisation** : Quel a été l'impact de GridSearch et de l'ajustement du seuil ?
4. **Déploiement** : Le modèle est-il prêt pour la production ? Quelles précautions ?
5. **Améliorations futures** : Que pourriez-vous faire pour améliorer encore le modèle ?